Convert audio file to .wav format if necessary


In [ ]:
import json
import pandas

excel_data_Enrol = pandas.read_excel('filename.xlsx', sheet_name='Enrolment')
excel_data_Acad = pandas.read_excel('filename.xlsx', sheet_name='Academics')

json_str_Enrol = excel_data_Enrol.to_json(orient='records')
json_str_Acad = excel_data_Acad.to_json(orient='records')

print('Enrolment Data to JSON:\n', json_str_Enrol, '\n\n')
print('Academic Data to JSON:\n', json_str_Acad, '\n\n')

data = {"analytics":[{"enrolment":json.loads(json_str_Enrol)},{"academics":json.loads(json_str_Acad)}]}
json_data = json.dumps(data)
# Write the data to a text file
with open('output.txt', 'w') as f:
        f.write(json_data)

Process generated .wav audio file and create fingerprint. Store in it fingerprints.txt


In [ ]:
from typing import List
from operator import xor

import pickle
import acoustid
import chromaprint

def get_fingerprint(filename: str) -> List[int]:
    """
    Returns:
        Returns a list of 32-bit integers () audio fingerprint. Two fingerprints can
        be roughly compared by counting the number of
        corresponding bits that are different from each other.
    """
    _ , encoded = acoustid.fingerprint_file(filename)
    fingerprint, _ = chromaprint.decode_fingerprint(
        encoded
    )
    return fingerprint

def fingerprint_distance(
    f1: List[int],
    f2: List[int],
    fingerprint_len: int,
) -> float:
    """
    Returns a normalized distance between two fingerprints.

    Args:
        f1: The first fingerprint.

        f2: The second fingerprint.

        fingerprint_len: Only compare the first `fingerprint_len`
            integers in each fingerprint. This is useful
            when comparing audio samples of a different length.

    Returns:
        Returns a number between 0.0 and 1.0 representing
        the distance between two fingerprints. This value
        represents distance as like a percentage.
    """
    max_hamming_weight = 32 * fingerprint_len
    hamming_weight = sum(
        sum(
            c == "1"
            for c in bin(xor(f1[i], f2[i]))
        )
        for i in range(fingerprint_len)
    )
    return hamming_weight / max_hamming_weight

# path to the audio file
audio_file_path = 'audio.wav'

# Generate fingerprint
fingerprint = get_fingerprint(audio_file_path)

# Store the fingerprint in a dictionary
fingerprint_dict = {
    'Artist': 'Sanchez',
    'Name': 'Mama (African Pride)',
    'fingerprint': fingerprint,
}

# Write the fingerprint dictionary to the signatures file
with open('signatures.txt', 'wb') as f:
    pickle.dump(fingerprint_dict, f)


Embed watermark in the audio file, store the new audio as audio_watermarked.wav. Update signatures.txt with the watermark data for that specific audio entry.